In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import json
import xgboost as xg
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from textblob import TextBlob
import torch
from torch import nn
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModel
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

/Users/abhishekvaidyanathan/Desktop/RedPILS/redpils/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Error loading vader_lexicon: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1129)>


device(type='cpu')

## Cleaning and splitting
Skip to bottom to read in final CSVs

In [2]:
full_data = pd.read_csv('full_data.csv')
full_data.head()

,submission_title,comment,comment_id,url,subreddit_id,label1,label2
0,Shashi Tharoor Set To Run For Congress Preside...,Extremely valid points but I believe he has th...,ip5g6vu,/r/india/comments/xif8wm/shashi_tharoor_set_to...,2qh1q,-1,-1
1,Yogi government puts Kolkata's image as part o...,Even if the ad was designed by the newspaper -...,hcontm8,/r/india/comments/pmn9o3/yogi_government_puts_...,2qh1q,-1,-1
2,"An attempt to address the list of ""simplified""...",Diverse population including Muslims. Welcomin...,fbhlv40,/r/india/comments/ebdeup/an_attempt_to_address...,2qh1q,-1,-2
3,You guys noticing what's happening in Sri Lanka?,>The blockade was done by your own people.\n\n...,i2yrud3,/r/india/comments/tt1ryh/you_guys_noticing_wha...,2qh1q,0,0
4,Just got abused on the train by a hyper-nation...,Ohoo bahut bura laga ye sunke ki aap Undergarm...,hooi92k,/r/india/comments/rh2kcs/just_got_abused_on_th...,2qh1q,-2,-2


In [3]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   submission_title  3000 non-null   object
 1   comment           3000 non-null   object
 2   comment_id        3000 non-null   object
 3   url               3000 non-null   object
 4   subreddit_id      3000 non-null   object
 5   label1            3000 non-null   int64 
 6   label2            3000 non-null   int64 
dtypes: int64(2), object(5)
memory usage: 164.2+ KB


In [4]:
def clean_quotations(text):
    temp = re.sub(r'>.*?\n', '', text)
    return temp
def clean_url(text):
    temp = re.sub(r'(http(s)?:\/\/.)?(www\.)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+\.~#?&\/=]*)', '', text)
    return temp
def clean_text(text):
    temp = clean_quotations(text)
    temp = clean_url(temp)
    temp = re.sub('\’', "\'", temp)  # apostrophe formatting is occasionally incorrect
    temp = re.sub(r'[^a-zA-Z0-9\s\.,?!\'\"]', ' ', temp)
    temp = re.sub(r'\s+', ' ', temp)  # replace multiple spaces with single space
    return temp.strip()

In [5]:
full_clean_df = pd.DataFrame()
full_clean_df['clean_title'] = full_data.apply(lambda x: clean_text(x['submission_title']), axis=1)
full_clean_df['clean_comment'] = full_data.apply(lambda x: clean_text(x['comment']), axis=1)
full_clean_df[['comment_id', 'url']] = full_data[['comment_id', 'url']].copy()
full_clean_df['target'] = full_data.apply(lambda x: np.mean([x['label1'], x['label2']])/2, axis=1)
# making sure to remove comments that are empty after cleaning
full_clean_df['clean_comment'].replace('', np.nan, inplace=True)
full_clean_df.dropna(inplace=True)
full_clean_df.head()

,clean_title,clean_comment,comment_id,url,target
0,Shashi Tharoor Set To Run For Congress Preside...,Extremely valid points but I believe he has th...,ip5g6vu,/r/india/comments/xif8wm/shashi_tharoor_set_to...,-0.50
1,Yogi government puts Kolkata's image as part o...,Even if the ad was designed by the newspaper t...,hcontm8,/r/india/comments/pmn9o3/yogi_government_puts_...,-0.50
2,"An attempt to address the list of ""simplified""...",Diverse population including Muslims. Welcomin...,fbhlv40,/r/india/comments/ebdeup/an_attempt_to_address...,-0.75
3,You guys noticing what's happening in Sri Lanka?,What a joke. They didn't create any propaganda...,i2yrud3,/r/india/comments/tt1ryh/you_guys_noticing_wha...,0.00
4,Just got abused on the train by a hyper nation...,Ohoo bahut bura laga ye sunke ki aap Undergarm...,hooi92k,/r/india/comments/rh2kcs/just_got_abused_on_th...,-1.00


In [6]:
full_clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2999 entries, 0 to 2999
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   clean_title    2999 non-null   object 
 1   clean_comment  2999 non-null   object 
 2   comment_id     2999 non-null   object 
 3   url            2999 non-null   object 
 4   target         2999 non-null   float64
dtypes: float64(1), object(4)
memory usage: 140.6+ KB


In [17]:
temp = full_clean_df.sample(frac=1, random_state=123).reset_index(drop=True)  # shuflling before split
eval_df = temp[:1000].copy()
train_df = temp[1000:].copy().reset_index(drop=True)

In [20]:
full_clean_df.to_csv('cleaned_full_data.csv', index=False)
train_df.to_csv('train_full.csv', index=False)
eval_df.to_csv('eval_full.csv', index=False)

In [7]:
# start here
train_df = pd.read_csv('train_full.csv')
eval_df = pd.read_csv('eval_full.csv')

## Rule-Based

In [8]:
right = """right wing, RW, authority, hierarchy, order, duty, tradition, reaction, nationalism, conservative, right-libertarian, \
neoconservative, imperialist, monarchist, fascist, reactionaries, traditionalist, traditional, death penalty, \
religion, Bhajpa, BJP, Shiv Sena, RSS, MNS, Sanatan, dharm, Hindutva, Islamophobia, Narendra, Modi, Amit, Shah, \
mandir, ram, valmiki, ramayan, Bharatiya, Janata, Democratic Alliance, NDA, AIADMK, Janta Dal, bhakt, CAA, NRC, hindu majority, \
hindu unity, hindu pride, nationalist, sangh, sanghi, yogi, brahmin, brahman, smriti irani, hindu rashtra, jai shri ram, \
pm cares, pmcares, adani, hindu""".lower()
left = """left wing, LW, leftists, freedom, equality, fraternity, rights, progress, reform, internationalism, anarchist, communist, socialist, \
democratic socialist, social democrat, left-libertarian, progressive, social, liberal, western, Congress, UPA, RG, mamata, \
Aam, aadmi, AAP, CPI, Welfare, Protectionism, Commies, Rahul, gandhi, indira, yatra, arvind, kejriwal, inclusivity, \
libby, libbies, sjw, libtard, hinduphobia, LGBTQ, masjid, pappu, christian, muslim, secular, minority, minorities, Shashi, Tharoor, \
gay, lesbian, transgender, trans, reservation, abrahamic, godi""".lower()

right_terms = set(right.split(', '))
left_terms = set(left.split(', '))
len(right_terms), len(left_terms)

(62, 60)

In [9]:
vader = SentimentIntensityAnalyzer()

In [10]:
def unsupervised_prediction(clean_text):
    # rule-based scoring
    sentiment = vader.polarity_scores(clean_text)['compound']
    lw = rw = 0
    clean_text = clean_text.lower()
    for t in left_terms:
        if re.search(r'\b'+t, clean_text):
            lw += 1
    for t in right_terms:
        if re.search(r'\b'+t, clean_text):
            rw += 1
    score = np.sign(rw - lw)*(abs(rw + lw)**0.05)*sentiment
    # print(sentiment)
    # normalize by length of text
    # score = np.nan_to_num(score)
    score /= len(clean_text.split())/20
    
    return score, sentiment

def conv_to_label(val, threshold=0.1):
    # converts rule-based score to label
    if abs(val)<threshold:
        return 0.
    else:
        return np.sign(val)

In [11]:
pred_df = eval_df.copy()
pred_df['true_label'] = pred_df.apply(lambda x: np.sign(x['target']), axis=1)
pred_df['vader_score'] = pred_df.apply(lambda x: unsupervised_prediction(x['clean_comment'])[0], axis=1)
pred_df['vader_sentiment'] = pred_df.apply(lambda x: unsupervised_prediction(x['clean_comment'])[1], axis=1)
pred_df['vader_label'] = pred_df.apply(lambda x: conv_to_label(x['vader_score'], threshold=0.01), axis=1)
pred_df.head()

,clean_title,clean_comment,comment_id,url,target,true_label,vader_score,vader_sentiment,vader_label
0,ResignModi trending in India on 1 with 200k tw...,"People tend to look down on ""social media acti...",gwe056n,/r/india/comments/n11sqc/resignmodi_trending_i...,-0.5,-1.0,-0.000000,-0.5594,0.0
1,"Put 'The Kashmir Files' on YouTube, everyone w...","I remember it was made to release twice, and t...",i1y3pv3,/r/india/comments/tmcun4/put_the_kashmir_files...,0.0,0.0,0.000000,0.7343,0.0
2,Kejriwal says 'The Kashmir Files' is a jhoothi...,Bhai delhi m hindu bht h but ye aur caste k lo...,i21haf8,/r/IndiaSpeaks/comments/tmhqqg/kejriwal_says_t...,0.5,1.0,0.320255,0.8807,1.0
3,Government of India has Blacklisted Karl Rock ...,mahatma gandhi's shadows are fading. retards w...,h4lazw7,/r/india/comments/ogrc0d/government_of_india_h...,-1.0,-1.0,-0.138984,-0.2960,-1.0
4,Kejriwal says 'The Kashmir Files' is a jhoothi...,Isn't this kejru the same guy who kept tweetin...,i1yrkwx,/r/IndiaSpeaks/comments/tmhqqg/kejriwal_says_t...,0.5,1.0,0.000000,0.4404,0.0


In [12]:
pred_df.describe()

,target,true_label,vader_score,vader_sentiment,vader_label
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.033750,-0.084000,-0.003991,0.024146,-0.024000
std,0.463965,0.761264,0.165419,0.641495,0.670726
min,-1.000000,-1.000000,-0.661680,-0.994100,-1.000000
25%,-0.250000,-1.000000,0.000000,-0.602550,0.000000
50%,0.000000,0.000000,-0.000000,0.027850,0.000000
75%,0.250000,1.000000,0.000000,0.632850,0.000000
max,1.000000,1.000000,0.560581,0.993600,1.000000


In [13]:
pred_df["vader_label"].value_counts()

 0.0    550
-1.0    237
 1.0    213
Name: vader_label, dtype: int64

In [14]:
np.mean(pred_df['true_label']==pred_df['vader_label'])  # 3-class accuracy

0.516

In [15]:
confusion_matrix(pred_df['true_label'], pred_df['vader_label'])

array([[120, 133,  82],
       [ 45, 317,  52],
       [ 72, 100,  79]])

In [16]:
print(classification_report(pred_df['true_label'], pred_df['vader_label'], target_names=['Left', 'Neutral', 'Right']))

              precision    recall  f1-score   support

        Left       0.51      0.36      0.42       335
     Neutral       0.58      0.77      0.66       414
       Right       0.37      0.31      0.34       251

    accuracy                           0.52      1000
   macro avg       0.48      0.48      0.47      1000
weighted avg       0.50      0.52      0.50      1000



<Axes: ylabel='Density'>

In [17]:
pred_df['target'].plot.kde()
pred_df['vader_score'].plot.kde()

In [18]:
y_rule_based = list(pred_df['vader_label']+1)

## Feature Extracted XGBoost

In [19]:
def feature_extraction(clean_text):
    clean_text = clean_text.lower()
    features = []
    for t in left_terms:
        features.append(len(re.findall(r'\b'+t, clean_text)))
    for t in right_terms:
        features.append(len(re.findall(r'\b'+t, clean_text)))
    
    return features

def add_sentiment_features(clean_text, extracted_feat):
    # appends sentiment scores onto extracted features
    temp = extracted_feat.copy()
    temp.append(vader.polarity_scores(clean_text)['compound'])
    tb = TextBlob(clean_text)
    temp.append(tb.sentiment.polarity)
    temp.append(tb.sentiment.subjectivity)
    return temp

In [20]:
# getting train embeddings
temp_df = train_df.copy()
temp_df['extracted_features'] = temp_df.apply(lambda x: feature_extraction(x['clean_comment']), axis=1)
# temp_df['final_features'] = temp_df.apply(lambda x: add_sentiment_features(x['clean_comment'], x['extracted_features']), axis=1)
X_train, y_train = list(temp_df['extracted_features']), [np.sign(x)+1 for x in list(temp_df['target'])]

# getting eval embeddings
temp_df = eval_df.copy()
temp_df['extracted_features'] = temp_df.apply(lambda x: feature_extraction(x['clean_comment']), axis=1)
# temp_df['final_features'] = temp_df.apply(lambda x: add_sentiment_features(x['clean_comment'], x['extracted_features']), axis=1)
X_test, y_test = list(temp_df['extracted_features']), [np.sign(x)+1 for x in list(temp_df['target'])]

print(len(X_train), len(y_train), len(X_test), len(y_test))

1999 1999 1000 1000


In [21]:
xgb = xg.XGBClassifier(n_estimators=100, max_depth=4, learning_rate=0.1)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
print("3-class classification accuracy:", np.mean([int(y_test[i]==y_pred[i]) for i in range(len(y_test))]))

3-class classification accuracy: 0.581


In [22]:
confusion_matrix(y_test, y_pred)

array([[151, 149,  35],
       [ 35, 361,  18],
       [ 70, 112,  69]])

In [23]:
print(classification_report(y_test, y_pred, target_names=['Left', 'Neutral', 'Right']))

              precision    recall  f1-score   support

        Left       0.59      0.45      0.51       335
     Neutral       0.58      0.87      0.70       414
       Right       0.57      0.27      0.37       251

    accuracy                           0.58      1000
   macro avg       0.58      0.53      0.53      1000
weighted avg       0.58      0.58      0.55      1000



In [24]:
y_feat_ext = y_pred

## Feature Extraction with PCA

In [25]:
pca = PCA(n_components=8)

# getting train embeddings
temp_df = train_df.copy()
temp_df['extracted_features'] = temp_df.apply(lambda x: feature_extraction(x['clean_comment']), axis=1)
temp_df['extracted_features'] = pca.fit_transform(list(temp_df['extracted_features'])).tolist()  # fit transform PCA
# temp_df['final_features'] = temp_df.apply(lambda x: add_sentiment_features(x['clean_comment'], x['extracted_features']), axis=1)
X_train, y_train = list(temp_df['extracted_features']), [np.sign(x)+1 for x in list(temp_df['target'])]

# getting eval embeddings
temp_df = eval_df.copy()
temp_df['extracted_features'] = temp_df.apply(lambda x: feature_extraction(x['clean_comment']), axis=1)
temp_df['extracted_features'] = pca.transform(list(temp_df['extracted_features'])).tolist()  # transform PCA without re-fitting
# temp_df['final_features'] = temp_df.apply(lambda x: add_sentiment_features(x['clean_comment'], x['extracted_features']), axis=1)
X_test, y_test = list(temp_df['extracted_features']), [np.sign(x)+1 for x in list(temp_df['target'])]

print(len(X_train), len(y_train), len(X_test), len(y_test))

1999 1999 1000 1000


In [26]:
xgb = xg.XGBClassifier(n_estimators=100, max_depth=4, learning_rate=0.01)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
print("3-class classification accuracy:", np.mean([int(y_test[i]==y_pred[i]) for i in range(len(y_test))]))

3-class classification accuracy: 0.565


In [27]:
confusion_matrix(y_test, y_pred)

array([[171, 118,  46],
       [ 58, 330,  26],
       [ 99,  88,  64]])

In [28]:
print(classification_report(y_test, y_pred, target_names=['Left', 'Neutral', 'Right']))

              precision    recall  f1-score   support

        Left       0.52      0.51      0.52       335
     Neutral       0.62      0.80      0.69       414
       Right       0.47      0.25      0.33       251

    accuracy                           0.56      1000
   macro avg       0.54      0.52      0.51      1000
weighted avg       0.55      0.56      0.54      1000



In [29]:
y_feat_pca = y_pred

## tf-idf word embeddings with XGBoost

In [30]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2), analyzer='word')

X_train = vectorizer.fit_transform(list(train_df['clean_comment']))
X_test = vectorizer.transform(list(eval_df['clean_comment']))
y_train, y_test = [np.sign(x)+1 for x in list(train_df['target'])], [np.sign(x)+1 for x in list(eval_df['target'])]

print(X_train.shape, X_test.shape)

(1999, 92418) (1000, 92418)


In [31]:
count = 0

new_voc = {}
for i in vectorizer.vocabulary_:
    if (i in left_terms) or (i in right_terms):
        new_voc[i] = count 
        count += 1 
print(new_voc)

{'order': 0, 'bjp': 1, 'aap': 2, 'congress': 3, 'shashi': 4, 'tharoor': 5, 'hindu': 6, 'modi': 7, 'hindutva': 8, 'rights': 9, 'caa': 10, 'fascist': 11, 'western': 12, 'trans': 13, 'nationalism': 14, 'christian': 15, 'religion': 16, 'freedom': 17, 'social': 18, 'muslim': 19, 'right wing': 20, 'minority': 21, 'amit': 22, 'shah': 23, 'bharatiya': 24, 'janata': 25, 'kejriwal': 26, 'nationalist': 27, 'ram': 28, 'secular': 29, 'upa': 30, 'gandhi': 31, 'nda': 32, 'rw': 33, 'duty': 34, 'cpi': 35, 'hindu rashtra': 36, 'pappu': 37, 'minorities': 38, 'equality': 39, 'reservation': 40, 'commies': 41, 'yogi': 42, 'liberal': 43, 'mandir': 44, 'yatra': 45, 'smriti irani': 46, 'conservative': 47, 'lgbtq': 48, 'nrc': 49, 'progressive': 50, 'adani': 51, 'hierarchy': 52, 'reaction': 53, 'rahul': 54, 'welfare': 55, 'gay': 56, 'brahmin': 57, 'rss': 58, 'progress': 59, 'left wing': 60, 'ramayan': 61, 'indira': 62, 'janta dal': 63, 'islamophobia': 64, 'sangh': 65, 'bhakt': 66, 'leftists': 67, 'aam': 68, 'aad

In [32]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2), analyzer='word', vocabulary=new_voc)

X_train = vectorizer.fit_transform(list(train_df['clean_comment']))
X_test = vectorizer.transform(list(eval_df['clean_comment']))
y_train, y_test = [np.sign(x)+1 for x in list(train_df['target'])], [np.sign(x)+1 for x in list(eval_df['target'])]

print(X_train.shape, X_test.shape)

(1999, 97) (1000, 97)


In [33]:
xgb = xg.XGBClassifier(n_estimators=100, max_depth=4, learning_rate=0.05)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
print("3-class classification accuracy:", np.mean([int(y_test[i]==y_pred[i]) for i in range(len(y_test))]))

3-class classification accuracy: 0.529


In [34]:
confusion_matrix(y_test, y_pred)

array([[115, 191,  29],
       [ 30, 367,  17],
       [ 59, 145,  47]])

In [35]:
print(classification_report(y_test, y_pred, target_names=['Left', 'Neutral', 'Right']))

              precision    recall  f1-score   support

        Left       0.56      0.34      0.43       335
     Neutral       0.52      0.89      0.66       414
       Right       0.51      0.19      0.27       251

    accuracy                           0.53      1000
   macro avg       0.53      0.47      0.45      1000
weighted avg       0.53      0.53      0.48      1000



In [36]:
y_w_tfidf = y_pred

## tf-idf word embeddings with PCA and sentiment
Since PCA doesn't work for sparse data, we use TruncatedSVD for the same effect

In [37]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2), analyzer='word', vocabulary=new_voc)
tsvd = TruncatedSVD(n_components=5, random_state=123)

temp_df = train_df.copy()
X_train = vectorizer.fit_transform(list(temp_df['clean_comment']))
temp_df['extracted_features'] = tsvd.fit_transform(X_train).tolist()
# temp_df['final_features'] = temp_df.apply(lambda x: add_sentiment_features(x['clean_comment'], x['extracted_features']), axis=1)
X_train, y_train = list(temp_df['extracted_features']), [np.sign(x)+1 for x in list(temp_df['target'])]

temp_df = eval_df.copy()
X_test = vectorizer.transform(list(temp_df['clean_comment']))
temp_df['extracted_features'] = tsvd.transform(X_test).tolist()
# temp_df['final_features'] = temp_df.apply(lambda x: add_sentiment_features(x['clean_comment'], x['extracted_features']), axis=1)
X_test, y_test = list(temp_df['extracted_features']), [np.sign(x)+1 for x in list(temp_df['target'])]

In [38]:
xgb = xg.XGBClassifier(n_estimators=100, max_depth=4, learning_rate=0.01)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
print("3-class classification accuracy:", np.mean([int(y_test[i]==y_pred[i]) for i in range(len(y_test))]))

3-class classification accuracy: 0.552


In [39]:
confusion_matrix(y_test, y_pred)

array([[160, 139,  36],
       [ 45, 346,  23],
       [ 93, 112,  46]])

In [40]:
print(classification_report(y_test, y_pred, target_names=['Left', 'Neutral', 'Right']))

              precision    recall  f1-score   support

        Left       0.54      0.48      0.51       335
     Neutral       0.58      0.84      0.68       414
       Right       0.44      0.18      0.26       251

    accuracy                           0.55      1000
   macro avg       0.52      0.50      0.48      1000
weighted avg       0.53      0.55      0.52      1000



In [41]:
y_tfidf_pca = y_pred

## tf-idf sub-word embeddings

In [42]:
vectorizer = TfidfVectorizer(ngram_range=(2, 10), analyzer='char_wb', vocabulary=new_voc)

X_train = vectorizer.fit_transform(list(train_df['clean_comment']))
X_test = vectorizer.transform(list(eval_df['clean_comment']))
y_train, y_test = [np.sign(x)+1 for x in list(train_df['target'])], [np.sign(x)+1 for x in list(eval_df['target'])]

print(X_train.shape, X_test.shape)

(1999, 97) (1000, 97)


In [43]:
xgb = xg.XGBClassifier(n_estimators=100, max_depth=4, learning_rate=0.05)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
print("3-class classification accuracy:", np.mean([int(y_test[i]==y_pred[i]) for i in range(len(y_test))]))

3-class classification accuracy: 0.569


In [44]:
confusion_matrix(y_test, y_pred)

array([[135, 167,  33],
       [ 33, 363,  18],
       [ 64, 116,  71]])

In [45]:
print(classification_report(y_test, y_pred, target_names=['Left', 'Neutral', 'Right']))

              precision    recall  f1-score   support

        Left       0.58      0.40      0.48       335
     Neutral       0.56      0.88      0.68       414
       Right       0.58      0.28      0.38       251

    accuracy                           0.57      1000
   macro avg       0.58      0.52      0.51      1000
weighted avg       0.57      0.57      0.54      1000



In [46]:
y_sw_tfidf = y_pred

## Data Augmentation with tf-idf/XGBoost

In [47]:
list_right_terms = list(right_terms)
list_left_terms = list(left_terms)
full_terms = left_terms.union(right_terms)

In [48]:
def random_swap(term):
    if term in left_terms:
        # LW term being swapped
        return np.random.choice(list_right_terms)
    elif term in right_terms:
        # RW term being swapped
        return np.random.choice(list_left_terms)
    print(f"Error with: {term}")

def swap_terms(text):
    term_match = re.compile('|'.join([r'\b'+t for t in full_terms]))
    
    temp = term_match.sub(lambda m: random_swap(m.group()), text.lower())
    return temp

In [167]:
aug_records = []
for i, row in train_df.iterrows():
    aug_records.append({'clean_comment':row['clean_comment'].lower(), 'target':row['target']})
    # adding in record with swapped terms and opposite score
    if(row['target']>0.0):
        aug_records.append({'clean_comment':swap_terms(row['clean_comment']), 'target':-row['target']})
        # aug_records.append({'clean_comment':swap_terms(row['clean_comment']), 'target':-row['target']})
aug_df = pd.DataFrame.from_records(aug_records)
aug_df

,clean_comment,target
0,there are two types of parties those that prom...,0.00
1,"if fundamentals are sound then, faith can help...",0.75
2,"if fundamentals are sound then, faith can help...",-0.75
3,"contrary to popular belief, until modi, india ...",-0.50
4,bro inhone bahut land hijack kiya hai kisse pa...,-0.25
...,...,...
2512,if you had spent 5 min looking at the actual s...,0.00
2513,he can look at education. that is something he...,0.00
2514,it is good a initiative but the plastic in roa...,0.00
2515,"yup, been reading about it since couple of wee...",0.00


In [168]:
vectorizer = TfidfVectorizer(ngram_range=(2, 10), analyzer='char_wb', vocabulary=new_voc)

X_train = vectorizer.fit_transform(list(aug_df['clean_comment']))
X_test = vectorizer.transform(list(eval_df['clean_comment']))
y_train, y_test = [np.sign(x)+1 for x in list(aug_df['target'])], [np.sign(x)+1 for x in list(eval_df['target'])]

print(X_train.shape, X_test.shape)

(2517, 97) (1000, 97)


In [172]:
xgb = xg.XGBClassifier(n_estimators=100, max_depth=4, learning_rate=0.1)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
print("3-class classification accuracy:", np.mean([int(y_test[i]==y_pred[i]) for i in range(len(y_test))]))

3-class classification accuracy: 0.562


In [173]:
confusion_matrix(y_test, y_pred)

array([[198, 108,  29],
       [ 89, 312,  13],
       [107,  92,  52]])

In [174]:
print(classification_report(y_test, y_pred, target_names=['Left', 'Neutral', 'Right']))

              precision    recall  f1-score   support

        Left       0.50      0.59      0.54       335
     Neutral       0.61      0.75      0.67       414
       Right       0.55      0.21      0.30       251

    accuracy                           0.56      1000
   macro avg       0.56      0.52      0.51      1000
weighted avg       0.56      0.56      0.54      1000



In [175]:
y_aug = y_pred

## Frozen Indian-sBERT embeddings with XGBoost
### Note: do NOT re-run this section without a powerful GPU. The pretrained model is massive.

In [56]:
model_name = "meedan/indian-sbert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)

# freezing everything
modules = [model.embeddings, *model.encoder.layer]
for module in modules:
    for param in module.parameters():
        param.requires_grad = False

In [57]:
# getting train embeddings from model
with torch.no_grad():
    model.eval()
    X_train = []
    batch_size = 16
    for i in range(0, len(train_df), batch_size):
        temp = tokenizer(list(train_df['clean_comment'][i:i+batch_size]), padding='max_length', max_length = 256, truncation=True, return_tensors="pt")
        X_train.extend(model(temp['input_ids'].to(device), temp['attention_mask'].to(device)).pooler_output.cpu().detach().numpy().tolist())
      
print(len(X_train), len(X_train[0]))

1999 768


In [58]:
# getting eval embeddings from model
with torch.no_grad():
    model.eval()
    X_test = []
    batch_size = 16
    for i in range(0, len(eval_df), batch_size):
        temp = tokenizer(list(eval_df['clean_comment'][i:i+batch_size]), padding='max_length', max_length = 256, truncation=True, return_tensors="pt")
        X_test.extend(model(temp['input_ids'].to(device), temp['attention_mask'].to(device)).pooler_output.cpu().detach().numpy().tolist())
      
print(len(X_test), len(X_test[0]))

1000 768


In [59]:
y_train, y_test = [np.sign(x)+1 for x in list(train_df['target'])], [np.sign(x)+1 for x in list(eval_df['target'])]

In [60]:
xgb = xg.XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.1)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
print("3-class classification accuracy:", np.mean([int(y_test[i]==y_pred[i]) for i in range(len(y_test))]))

3-class classification accuracy: 0.518


In [61]:
confusion_matrix(y_test, y_pred)

array([[149, 123,  63],
       [ 82, 296,  36],
       [ 77, 101,  73]], dtype=int64)

In [62]:
print(classification_report(y_test, y_pred, target_names=['Left', 'Neutral', 'Right']))

              precision    recall  f1-score   support

        Left       0.48      0.44      0.46       335
     Neutral       0.57      0.71      0.63       414
       Right       0.42      0.29      0.35       251

    accuracy                           0.52      1000
   macro avg       0.49      0.48      0.48      1000
weighted avg       0.50      0.52      0.50      1000



In [63]:
y_bert_frozen = y_pred

## Ensemble of Models

In [ ]:
# we have the models and accuracies:
# y_rule_based    0.502
# y_feat_ext      0.574
# y_feat_pca      0.562
# y_w_tfidf       0.528
# y_tfidf_pca     0.493
# y_sw_tfidf      0.563
# y_aug           0.541
# y_bert_frozen   0.518

In [176]:
def majority_voting(pred_list):
    # provide in decreasing order of confidence for tie-breaks
    n = len(pred_list[0])
    y_ensembled = []
    for i in range(n):
        temp = [x[i] for x in pred_list]
        y_ensembled.append(Counter(temp).most_common(1)[0][0])
    return y_ensembled

In [177]:
# y_ensembled = majority_voting([y_feat_ext, y_sw_tfidf, y_feat_pca, y_aug, y_w_tfidf, y_tfidf_pca])
y_ensembled = majority_voting([y_feat_ext, y_sw_tfidf, y_feat_pca, y_aug])
y_test = [np.sign(x)+1 for x in list(eval_df['target'])]
y_pred = y_ensembled
print("3-class classification accuracy:", np.mean([int(y_test[i]==y_pred[i]) for i in range(len(y_test))]))

3-class classification accuracy: 0.58


In [178]:
confusion_matrix(y_test, y_pred)

array([[152, 149,  34],
       [ 41, 357,  16],
       [ 71, 109,  71]])

In [179]:
print(classification_report(y_test, y_pred, target_names=['Left', 'Neutral', 'Right']))

              precision    recall  f1-score   support

        Left       0.58      0.45      0.51       335
     Neutral       0.58      0.86      0.69       414
       Right       0.59      0.28      0.38       251

    accuracy                           0.58      1000
   macro avg       0.58      0.53      0.53      1000
weighted avg       0.58      0.58      0.55      1000



In [180]:
y_majority = y_pred

In [181]:
def weighted_avg(pred_list):
    # pred_list should contain tuples of (y_pred, weight)
    n = len(pred_list[0][0])
    y_ensembled = []
    weight_sum = sum([m[1] for m in pred_list])
    for i in range(n):
        score = 0
        for m in pred_list:
            score += m[0][i]*m[1]/weight_sum
        y_ensembled.append(score-1)
    return y_ensembled

In [182]:
y_ensembled = weighted_avg([(y_feat_ext, 10), 
                           (y_sw_tfidf, 10), 
                           (y_feat_pca,8), 
                           (y_aug, 7), 
                           # (y_w_tfidf, 20), 
                        #    (y_bert_frozen, 0), 
                           (y_rule_based, 0), 
                           (y_tfidf_pca, 0)])

(array([186.,  22.,  78.,  95.,  11., 492.,  21.,  32.,   9.,  54.]),
 array([-1. , -0.8, -0.6, -0.4, -0.2,  0. ,  0.2,  0.4,  0.6,  0.8,  1. ]),
 <BarContainer object of 10 artists>)

In [183]:
plt.hist(y_ensembled)

In [184]:
y_pred = [conv_to_label(x, threshold=0.1)+1 for x in y_ensembled]
# y_pred = [1 if -0.25<x<0.15 else np.sign(x)+1 for x in y_ensembled]
print("3-class classification accuracy:", np.mean([int(y_test[i]==y_pred[i]) for i in range(len(y_test))]))

3-class classification accuracy: 0.591


In [185]:
confusion_matrix(y_test, y_pred)

array([[209,  93,  33],
       [ 86, 308,  20],
       [ 89,  88,  74]])

In [186]:
print(classification_report(y_test, y_pred, target_names=['Left', 'Neutral', 'Right']))

              precision    recall  f1-score   support

        Left       0.54      0.62      0.58       335
     Neutral       0.63      0.74      0.68       414
       Right       0.58      0.29      0.39       251

    accuracy                           0.59      1000
   macro avg       0.59      0.55      0.55      1000
weighted avg       0.59      0.59      0.58      1000



In [ ]:
# normal bert fine-tuning, aug+bert, lstm

## Propogating results of ensemble